#### 同策略基金分析

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pyecharts.charts import Tab
from pyecharts.components import Table
from function import *
from WindPy import w
from pathlib import Path

w.start()

import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
plt.rcParams["font.sans-serif"] = ["Microsoft YaHei"]
plt.rcParams["axes.unicode_minus"] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
# 辅助日期
year_begin_date = "2024-12-27"
month_begin_date = "2025-01-24"
week_begin_date = "2025-02-07"

In [3]:
basic_info = load_data("产品目录.xlsx")
basic_info

,大类策略,细分策略,基金代码,基金名称,基准代码,基准名称,最大回撤阈值,杠杆率
0,主观CTA,时序,SGC034,济海实恒对冲二号,NH0100.NHF,南华商品指数,-0.10,NaN
1,主观CTA,复合,SM6151,草本优益1号,NH0100.NHF,南华商品指数,-0.05,NaN
2,主观CTA,复合,SM6151,草本致远1号,NH0300.NHF,南华农产品指数,-0.10,NaN
3,主观CTA,复合,ST6571,星恒哲萌1号,NH0100.NHF,南华商品指数,-0.05,NaN
4,量化CTA,截面,SS0221,均成CTA1号,NH0100.NHF,南华商品指数,-0.10,NaN
5,量化CTA,时序,SD8726,黑翼CTA三号,NH0100.NHF,南华商品指数,-0.05,NaN
6,量化CTA,时序,SW8015,旭诺CTA三号,NH0100.NHF,南华商品指数,-0.05,平均25%
7,量化CTA,时序,SXF557,千衍九凌1号,NH0100.NHF,南华商品指数,-0.03,NaN
8,量化CTA,时序,SCV067,众壹资产量合兴成一号,NH0100.NHF,南华商品指数,-0.05,NaN
9,量化CTA,时序,SLS182,弘源多元化CTA,NH0100.NHF,南华商品指数,-0.05,NaN


#### 表格

In [4]:
nav_dfs = None
nav_table = pd.DataFrame(
    columns=[
        "大类策略",
        "细分策略",
        "产品名称",
        "日期",
        "单位净值",
        "累计净值",
        "复权净值",
        "本年收益(%)",
        "当月收益(%)",
        "当周收益(%)",
    ]
)
for fund in basic_info["基金名称"]:
    # 导入净值数据
    nav_df_path = pd.Series(
        [
            i
            for i in Path("./data").rglob("*")
            if (i.suffix.lower() in {".csv", ".xlsx", ".xls"}) & (fund in i.name)
        ]
    )
    nav_df = load_data(nav_df_path.item())
    nav_df = get_standardized_data(nav_df)
    nav_df = get_nav_adjusted(nav_df)
    nav_df.rename(
        columns={
            "date": "日期",
            "nav_unit": "单位净值",
            "nav_accumulated": "累计净值",
            "nav_adjusted": "复权净值",
        },
        inplace=True,
    )
    # 计算年、月、周收益
    year_begin_nav = nav_df[nav_df["日期"] == year_begin_date]
    month_begin_nav = nav_df[nav_df["日期"] == month_begin_date]
    week_begin_nav = nav_df[nav_df["日期"] == week_begin_date]
    nav_head = nav_df.head(1)
    nav_tail = nav_df.tail(1)
    nav_tail["产品名称"] = fund
    nav_tail["大类策略"] = basic_info[basic_info["基金名称"] == fund]["大类策略"].item()
    nav_tail["细分策略"] = basic_info[basic_info["基金名称"] == fund]["细分策略"].item()
    if not year_begin_nav.empty:
        year_begin_value = year_begin_nav["复权净值"].iloc[0]
        nav_tail["本年收益(%)"] = round(
            ((nav_tail["复权净值"].iloc[0] / year_begin_value) - 1) * 100, 2
        )
    else:  # 如果找不到2023-12-29净值，则使用第一条数据
        nav_tail["本年收益(%)"] = round(
            ((nav_tail["复权净值"].iloc[0] / nav_head["复权净值"].iloc[0]) - 1) * 100, 2
        )
    month_begin_value = month_begin_nav["复权净值"].iloc[0]
    nav_tail["当月收益(%)"] = round(
        ((nav_tail["复权净值"].iloc[0] / month_begin_value) - 1) * 100, 2
    )
    week_begin_value = week_begin_nav["复权净值"].iloc[0]
    nav_tail["当周收益(%)"] = round(
        ((nav_tail["复权净值"].iloc[0] / week_begin_value) - 1) * 100, 2
    )
    # 添加至表格
    nav_table = pd.concat([nav_table, nav_tail], ignore_index=True)
    custom_order = ["主观CTA", "量化CTA", "套利", "期权"]
    custom_order_ = ["趋势", "截面", "复合", "套利", "期权"]
    nav_table["大类策略"] = pd.Categorical(
        nav_table["大类策略"], categories=custom_order, ordered=True
    )
    nav_table["细分策略"] = pd.Categorical(
        nav_table["细分策略"], categories=custom_order_, ordered=True
    )
    nav_table.sort_values(
        by=["大类策略", "细分策略", "本年收益(%)"],
        inplace=True,
        ascending=[True, True, False],
        ignore_index=True,
    )
nav_table["日期"] = nav_table["日期"].map(lambda x: x.strftime("%Y-%m-%d"))

正在处理济海实恒对冲二号
正在处理草本优益1号
正在处理草本致远1号
正在处理星恒哲萌1号
正在处理均成CTA1号
正在处理黑翼CTA三号
正在处理旭诺CTA三号
正在处理千衍九凌1号
正在处理众壹资产量合兴成一号
正在处理弘源多元化CTA
正在处理会世元丰CTA1号
正在处理远澜云杉
正在处理量派CTA七号
正在处理安贤CTA量化一号
正在处理远澜翠柏1号
正在处理旌安1号
正在处理殊馥馥源套利1号


In [6]:
table_index = Table()
headers = list(nav_table.columns)
rows = nav_table.values
table_index.add(headers, rows)

In [7]:
nav_table

,大类策略,细分策略,产品名称,日期,单位净值,累计净值,复权净值,本年收益(%),当月收益(%),当周收益(%)
0,主观CTA,复合,草本优益1号,2025-02-14,1.0510,3.6130,5.789511,4.68,4.16,1.74
1,主观CTA,复合,星恒哲萌1号,2025-02-14,1.1219,1.5852,1.702920,0.63,0.35,0.28
2,主观CTA,复合,草本致远1号,2025-02-14,1.0980,3.0710,4.579741,-1.96,9.47,8.82
3,主观CTA,NaN,济海实恒对冲二号,2025-02-07,1.0610,1.9570,2.260265,1.05,1.63,0.00
4,量化CTA,截面,均成CTA1号,2025-02-14,6.6660,7.3110,7.665900,0.08,2.92,0.59
5,量化CTA,复合,会世元丰CTA1号,2025-02-14,2.0026,2.0026,2.002600,0.25,2.08,0.34
6,量化CTA,复合,远澜云杉,2025-02-14,2.8460,2.8460,2.846000,-0.32,-0.11,-0.04
7,量化CTA,NaN,千衍九凌1号,2025-02-14,1.3796,1.2581,1.246900,3.79,0.87,0.37
8,量化CTA,NaN,旭诺CTA三号,2025-02-14,3.8075,3.8075,3.807500,2.19,-1.31,-1.70
9,量化CTA,NaN,黑翼CTA三号,2025-02-14,3.1081,3.1081,3.108100,0.82,1.77,0.40


In [11]:
print("主观CTA平均收益:",round(nav_table[nav_table["大类策略"] == "主观CTA"]["当周收益(%)"].mean(),2))
print("量化CTA平均收益:",round(nav_table[nav_table["大类策略"] == "量化CTA"]["当周收益(%)"].mean(),2))
print("套利平均收益:",round(nav_table[nav_table["大类策略"] == "套利"]["当周收益(%)"].mean(),2))

主观CTA平均收益: 2.71
量化CTA平均收益: -0.14
套利平均收益: 0.57


#### 折线图

In [17]:
def get_data(strategy):
    strategy_info = basic_info.loc[(basic_info["大类策略"] == strategy), :]
    strategy_info.reset_index(drop=True, inplace=True)
    nav_dfs = None
    for fund in strategy_info["基金名称"]:
        nav_df_path = pd.Series(
        [
            i
            for i in Path("./data").rglob("*")
            if (i.suffix.lower() in {".csv", ".xlsx", ".xls"}) & (fund in i.name)
        ]
    )
        nav_df = load_data(nav_df_path.item())
        nav_df = get_standardized_data(nav_df)
        nav_df = get_nav_adjusted(nav_df)
        nav_df.rename(columns={"date": "日期", "nav_adjusted": fund}, inplace=True)
        nav_df = nav_df[["日期", fund]]
        if nav_dfs is None:
            nav_dfs = nav_df
        else:
            nav_dfs = pd.merge(nav_dfs, nav_df, on="日期", how="inner")
            nav_dfs = nav_dfs.set_index("日期")
            nav_dfs = nav_dfs.loc["2023-12-29":]
            nav_dfs = nav_dfs.div(nav_dfs.iloc[0])
    return nav_dfs

In [19]:
get_line(get_data("主观CTA"), "主观CTA").render_notebook()

In [20]:
get_line(get_data("量化CTA"), "量化CTA").render_notebook()

In [21]:
get_line(get_data("套利"), "套利").render_notebook()

In [22]:
tab = Tab()
tab.add(table_index, "各策略CTA产品涨跌幅")
tab.add(get_line(get_data("主观CTA"), "主观CTA"), "主观CTA走势")
tab.add(get_line(get_data("量化CTA"), "量化CTA"), "量化CTA走势")
tab.add(get_line(get_data("套利"), "套利"), "套利")
tab.render("CTA策略收益跟踪.html")

'e:\\桌面文件\\Vscode\\Private_nav_research\\CTA策略收益跟踪.html'